In [2]:
import re  # For regular expressions (if needed)

import pandas as pd

from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [19]:
qa = pd.read_csv("dataset/train.csv")
qa

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


In [20]:
# dataframe = pd.concat([df, qa, qa2], ignore_index=True)
# dataframe = dataframe.dropna(axis=1, how='all')

# dataframe

In [21]:
# # Define the output file path
# output_file = "dataset/dataset.txt"

# # Open file in write mode
# with open(output_file, "w", encoding="utf-8") as f:
#     for index, row in dataframe.iterrows():
#         f.write(f"user: {row['user']}\n")
#         f.write(f"bot: {row['bot']}\n\n")  # Add an empty line for readability

# print(f"File saved as {output_file}")

In [1]:
import os
import random

seed = 1337
random.seed(seed)

# Define file paths
dataset_dir = "dataset"
file_path = os.path.join(dataset_dir, "dataset.txt")
train_file_path = os.path.join(dataset_dir, "train.txt")
validation_file_path = os.path.join(dataset_dir, "validation.txt")

# Delete existing train and validation files if they exist
for file in [train_file_path, validation_file_path]:
    if os.path.exists(file):
        os.remove(file)

# Load the dataset
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Ensure alternating user-bot pairs
dialogues = []
current_dialogue = []

for line in lines:
    line = line.strip()
    if line:
        current_dialogue.append(line)
        if len(current_dialogue) == 2:
            dialogues.append(current_dialogue)
            current_dialogue = []

# Shuffle and split the data into 80% train and 20% validation
random.shuffle(dialogues)
split_index = int(0.8 * len(dialogues))

train_dialogues = dialogues[:split_index]
validation_dialogues = dialogues[split_index:]

# Write to train.txt
with open(train_file_path, "w", encoding="utf-8") as train_file:
    for user, bot in train_dialogues:
        train_file.write(f"{user}\n{bot}\n")

# Write to validation.txt
with open(validation_file_path, "w", encoding="utf-8") as validation_file:
    for user, bot in validation_dialogues:
        validation_file.write(f"{user}\n{bot}\n")

print(f"Train file saved at: {train_file_path}")
print(f"Validation file saved at: {validation_file_path}")

Train file saved at: dataset\train.txt
Validation file saved at: dataset\validation.txt


In [17]:
import json
import os 
from dotenv import load_dotenv
from google import genai

# Configure Gemini API
load_dotenv()
API_KEY = os.getenv("API_KEY")

# Load JSON dataset
json_file_path = "dataset/intents.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Convert dataset into structured text for Gemini
formatted_data = ""
for intent in data["intents"]:
    patterns = " | ".join(intent["patterns"])  # Join patterns in a readable format
    responses = " | ".join(intent["responses"])  # Join responses similarly
    formatted_data += f"\nUser inputs: {patterns}\nBot responses: {responses}\n\n"

# Define the structured prompt including the full dataset
prompt = f"""
I have a chatbot dataset with multiple user intents. Each intent consists of:
- A list of user input examples (patterns).
- A list of possible bot responses.

Below is the full dataset:

{formatted_data}

Your task:
1. For each intent, pick 4 **random user input** as the user message.
2. Pick 4 **random bot response** as the bot reply.
3. Format the conversation as follows:

User: <random user input>
Bot: <random bot response>

Generate a full conversation covering all intents.
"""

client = genai.Client(api_key=API_KEY)
response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=prompt
)

print(response.text)


Okay, here's a simulated conversation based on the provided dataset, with 4 random user inputs and 4 random bot responses for each intent:

**1. Greeting**

User: Guten tag
Bot: Hello there. How are you feeling today?

User: Ola
Bot: Hi there. Tell me how are you feeling today?

User: Hi there
Bot: Hello there. Tell me how are you feeling today?

User: Howdy
Bot: Hello there. Glad to see you're back. What's going on in your world right now?

**2. Good Morning**

User: Good morning
Bot: Good morning. I hope you had a good night's sleep. How are you feeling today? 

User: Good morning
Bot: Good morning. I hope you had a good night's sleep. How are you feeling today? 

User: Good morning
Bot: Good morning. I hope you had a good night's sleep. How are you feeling today? 

User: Good morning
Bot: Good morning. I hope you had a good night's sleep. How are you feeling today? 

**3. Good Afternoon**

User: Good afternoon
Bot: Good afternoon. How is your day going?

User: Good afternoon
Bot: Go

hello

In [1]:
import os 

text_data = []

# Define file paths
train_file = "dataset/train.txt"
val_file = "dataset/val.txt"

# Check if files exist and delete them
for file in [train_file, val_file]:
    if os.path.exists(file):
        os.remove(file)
        print("file removed")

# Ensure dataset directory exists
os.makedirs("dataset", exist_ok=True)

for i in range(0, len(qa)):
    user_utterance = qa['Context'][i] # Access user utterance using column name
    bot_utterance = qa['Response'][i] # Access bot utterance using column name

    # Handle missing values (if any)
    if pd.isna(user_utterance):
        user_utterance = "User: "  # Or some other placeholder
    if pd.isna(bot_utterance):
        bot_utterance = "Bot: I don't know."  # Or a default response

    text_data.append(f"User: {user_utterance}")
    text_data.append(f"Bot: {bot_utterance}")

with open("dataset/text.txt", "w", encoding="utf-8") as f:
    for line in text_data:
        f.write(line + "\n")
        
# # 3. Split into training and validation sets
# train_data = text_data[:int(len(text_data)*0.8)]  # 80% for training
# val_data = text_data[int(len(text_data)*0.8):]  # 20% for validation

# # 4. Save to text files (train.txt and val.txt)
# with open("dataset/train.txt", "w", encoding="utf-8") as f: # Add encoding for special characters
#     for line in train_data:
#         f.write(line + "\n")

# with open("dataset/val.txt", "w", encoding="utf-8") as f: # Add encoding for special characters
#     for line in val_data:
#         f.write(line + "\n")

# ... (rest of your fine-tuning code using train.txt and val.txt)

file removed


NameError: name 'qa' is not defined